In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
# Generate dummy data
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [3]:
# Define the MLP model
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_layers, activation_function):
        super(MLPModel, self).__init__()
        layers = []
        current_dim = input_dim
        
        # Add hidden layers
        for hidden_neurons in hidden_layers:
            layers.append(nn.Linear(current_dim, hidden_neurons))
            if activation_function == 'relu':
                layers.append(nn.ReLU())
            elif activation_function == 'sigmoid':
                layers.append(nn.Sigmoid())
            elif activation_function == 'tanh':
                layers.append(nn.Tanh())
            elif activation_function == 'linear':
                pass  # No activation for linear
            elif activation_function == 'softmax':
                layers.append(nn.Softmax(dim=1))
            current_dim = hidden_neurons
        
        # Add output layer
        layers.append(nn.Linear(current_dim, 1))
        layers.append(nn.Sigmoid())  # Binary classification
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)


In [4]:
# Define hyperparameters
# Hyperparameter grid
hidden_layers_options = [
    [4], [8], [16], [32], [64],    # 1 hidden layer
    [4, 4], [8, 8], [16, 16], [32, 32], [64, 64],  # 2 hidden layers
    [4, 8, 4], [8, 16, 8], [16, 32, 16], [32, 64, 32], [64, 128, 64] # 3 hidden layers
]
activation_options = ['linear', 'sigmoid', 'relu', 'softmax', 'tanh']
epoch_options = [1, 10, 25, 50, 100, 250]
learning_rate_options = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_size_options = [16, 32, 64, 128, 256, 512]

# Save configuration for progress tracking
results = []

# Function to evaluate model
def train_and_evaluate(hidden_layers, activation_function, epochs, learning_rate, batch_size):
    # Create DataLoader for training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model, loss, and optimizer
    model = MLPModel(input_dim=X_train_tensor.shape[1], hidden_layers=hidden_layers, activation_function=activation_function)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    epoch_losses = []
    for epoch in range(epochs):
        total_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        # Calculate average loss for this epoch
        avg_loss = total_loss / len(train_loader)
        epoch_losses.append(avg_loss)
    
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in train_loader:  # Using the entire dataset for evaluation
            predictions = model(batch_X)
            predicted = (predictions > 0.5).float()
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)
    accuracy = correct / total
    return accuracy, epoch_losses

# Running the grid search for hyperparameters
for hidden_layers in hidden_layers_options:
    for activation in activation_options:
        for epochs in epoch_options:
            for lr in learning_rate_options:
                for batch_size in batch_size_options:
                    # Train and evaluate the model
                    accuracy, losses = train_and_evaluate(
                        hidden_layers=hidden_layers,
                        activation_function=activation,
                        epochs=epochs,
                        learning_rate=lr,
                        batch_size=batch_size
                    )
                    
                    # Store results
                    results.append({
                        'hidden_layers': hidden_layers,
                        'activation': activation,
                        'epochs': epochs,
                        'learning_rate': lr,
                        'batch_size': batch_size,
                        'accuracy': accuracy,
                        'loss': losses  
                    })
                    print("============================================")
                    print(f"HL: {hidden_layers}, Act: {activation}, Epochs: {epochs}, LR: {lr}, BS: {batch_size}, Accuracy: {accuracy:.4f}, Loss: {losses[-1]:.4f}")    
                    print("============================================")

# Save the results to a CSV file
data = pd.DataFrame(results)
data.to_csv('MLP_Classification_DummyData.csv', index=False)


HL: [4], Act: linear, Epochs: 1, LR: 10, BS: 16, Accuracy: 0.3762, Loss: 61.4816
HL: [4], Act: linear, Epochs: 1, LR: 10, BS: 32, Accuracy: 0.3463, Loss: 62.2296
HL: [4], Act: linear, Epochs: 1, LR: 10, BS: 64, Accuracy: 0.6675, Loss: 50.1614
HL: [4], Act: linear, Epochs: 1, LR: 10, BS: 128, Accuracy: 0.6275, Loss: 50.9032
HL: [4], Act: linear, Epochs: 1, LR: 10, BS: 256, Accuracy: 0.3162, Loss: 53.9266
HL: [4], Act: linear, Epochs: 1, LR: 10, BS: 512, Accuracy: 0.4788, Loss: 8.9347
HL: [4], Act: linear, Epochs: 1, LR: 1, BS: 16, Accuracy: 0.8275, Loss: 16.8594
HL: [4], Act: linear, Epochs: 1, LR: 1, BS: 32, Accuracy: 0.8438, Loss: 5.2079
HL: [4], Act: linear, Epochs: 1, LR: 1, BS: 64, Accuracy: 0.8100, Loss: 8.1827
HL: [4], Act: linear, Epochs: 1, LR: 1, BS: 128, Accuracy: 0.8350, Loss: 8.3693
HL: [4], Act: linear, Epochs: 1, LR: 1, BS: 256, Accuracy: 0.7475, Loss: 6.6052
HL: [4], Act: linear, Epochs: 1, LR: 1, BS: 512, Accuracy: 0.5950, Loss: 2.1121
HL: [4], Act: linear, Epochs: 1, L

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load hasil eksperimen dari file CSV
data = pd.read_csv("MLP_Classification_DummyData.csv")

# Buat folder untuk menyimpan hasil plot jika belum ada
output_folder = "mlp_experiment_DummyData_plots"
os.makedirs(output_folder, exist_ok=True)

# Hyperparameter grid untuk visualisasi
hidden_layers_list = [
    str([4]), str([8]), str([16]), str([32]), str([64]),  # 1 hidden layer
    str([4, 4]), str([8, 8]), str([16, 16]), str([32, 32]), str([64, 64]),  # 2 hidden layers
    str([4, 8, 4]), str([8, 16, 8]), str([16, 32, 16]), str([32, 64, 32]),  # 3 hidden layers
]
activations_list = ['linear', 'sigmoid', 'relu', 'softmax', 'tanh']
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]

# Loop melalui kombinasi parameter
for hidden_layers in hidden_layers_list:
    for activation in activations_list:
        for lr in learning_rates:
            # Filter data untuk kombinasi parameter ini
            filtered_data = data[
                (data['hidden_layers'] == hidden_layers) & 
                (data['activation'] == activation) & 
                (data['learning_rate'] == lr)
            ]
            
            if not filtered_data.empty:
                # Plot akurasi terhadap epochs untuk setiap batch size
                batch_sizes = filtered_data['batch_size'].unique()

                plt.figure(figsize=(12, 12))
                for i, batch_size in enumerate(batch_sizes, 1):
                    plt.subplot(3, 3, i)  # Buat grid 3x3
                    batch_data = filtered_data[filtered_data['batch_size'] == batch_size]
                    plt.plot(batch_data['epochs'], batch_data['accuracy'], marker='o', label=f'Batch Size: {batch_size}')
                    plt.title(f'HL: {hidden_layers}, Act: {activation}, LR: {lr}, BS: {batch_size}')
                    plt.xlabel('Epochs')
                    plt.ylabel('Accuracy')
                    plt.legend()
                    plt.grid()

                plt.tight_layout()
                
                # Nama file berdasarkan kombinasi hyperparameter
                filename = f"HL-{hidden_layers}_Act-{activation}_LR-{lr}.png"
                filepath = os.path.join(output_folder, filename)
                
                # Simpan plot ke file
                plt.savefig(filepath, dpi=150)
                plt.close() 


In [6]:
import pandas as pd

# Load the experiment results from the CSV file
results_df = pd.read_csv("MLP_Classification_DummyData.csv")

# Sort the results by accuracy in descending order
top_results = results_df.sort_values(by="accuracy", ascending=False).head(10)

# Display the top 10 results
top_results


,hidden_layers,activation,epochs,learning_rate,batch_size,accuracy,loss
15072,"[32, 64, 32]",tanh,100,0.001,16,1.0,"[0.5654052084684372, 0.344541600048542, 0.3169..."
4305,[32],tanh,250,0.010,128,1.0,"[0.6447941660881042, 0.47728934458323885, 0.37..."
4307,[32],tanh,250,0.010,512,1.0,"[0.6742006242275238, 0.6000317633152008, 0.545..."
4308,[32],tanh,250,0.001,16,1.0,"[0.6243017649650574, 0.511233206987381, 0.4281..."
3651,[32],sigmoid,250,0.100,128,1.0,"[0.5797000016484942, 0.3485583130802427, 0.390..."
3650,[32],sigmoid,250,0.100,64,1.0,"[0.45903376661814177, 0.3509198736685973, 0.32..."
3649,[32],sigmoid,250,0.100,32,1.0,"[0.4850424927473068, 0.3280319267511368, 0.325..."
3648,[32],sigmoid,250,0.100,16,1.0,"[0.44591430634260176, 0.34889720402657987, 0.3..."
3647,[32],sigmoid,250,1.000,512,1.0,"[4.277841329574585, 2.6835979223251343, 7.1772..."
9197,"[32, 32]",relu,50,0.100,512,1.0,"[0.5938630998134613, 0.5269571542739868, 0.366..."
